# Lab 1: Let's build a forest! 

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
# import warnings
# warnings.filterwarnings('ignore')
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split


numerical = pd.read_csv('numerical.csv')
categorical = pd.read_csv('categorical.csv')
targets = pd.read_csv('target.csv')
data = pd.concat([numerical, categorical, targets], axis = 1)
data['TARGET_B'].value_counts()

0    90569
1     4843
Name: TARGET_B, dtype: int64

In [2]:
data.columns[data.isna().any()].tolist()

[]

In [3]:
#No NaNs! 

In [4]:
#Then we split and encode our data

In [5]:
y = data['TARGET_B']
X = data.drop(['TARGET_B'], axis = 1)

numericalX = X.select_dtypes(np.number)
categoricalX = X.select_dtypes(np.object)

from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first').fit(categoricalX)
encoded_categorical = encoder.transform(categoricalX).toarray()
encoded_categorical = pd.DataFrame(encoded_categorical)
X = pd.concat([numericalX, encoded_categorical], axis = 1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

/var/folders/4x/ppplmryx5_d7mfrd5d6968kr0000gn/T/ipykernel_71349/393626257.py:5: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  categoricalX = X.select_dtypes(np.object)


In [6]:
trainset = pd.concat([X_train, y_train], axis=1)

category_0 = trainset[trainset['TARGET_B']==0]
print(category_0.shape)

from sklearn.utils import resample
cat_1 = trainset[trainset['TARGET_B']== 1 ]
category_1 = resample(cat_1, replace = True, n_samples = len(category_0))
trainset_new = pd.concat([category_0, category_1], axis = 0)
trainset_new = trainset_new.sample(frac =1)
X_train = trainset_new.drop(['TARGET_B'], axis=1)
y_train = trainset_new['TARGET_B']

print(X_train.shape)

(72486, 356)
(144972, 355)


In [7]:
#Now we have an upsampled dataset so we can model. 

In [8]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

y_train_regression = X_train['TARGET_D']
y_test_regression = X_test['TARGET_D']

X_train = X_train.drop(['TARGET_D'], axis = 1)
X_test = X_test.drop(['TARGET_D'], axis = 1)

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20,
                             max_samples=0.8,
                             random_state = 42)
clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

y_pred = clf.predict(X_test)
display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred))

/Users/lex/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/lex/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


0.6201818282151036
0.6070324372478122


/Users/lex/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/lex/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


0    18083
1     1000
Name: TARGET_B, dtype: int64

array([[11039,  7044],
       [  455,   545]])

In [10]:
#Upsampling the data gives us a slightly better result than the downsampling done in class, but the margin is very small
#Given the instability of "Tree based" models (even forests), I wouldn't use a difference of 0.02 to make a final decision on which way is better
#We need cross validation to get a more robust result. 

In [11]:
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20)
cross_val_scores = cross_val_score(clf, X_train, y_train, cv=10)
print(np.mean(cross_val_scores))


/Users/lex/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/lex/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/lex/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/lex/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  war

0.616643286270606


/Users/lex/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [12]:
#Cross validation result of downsampling was 0.58, we now have 0.62. It is a small margin, but I would recommend to go with upsampling.

# Lab 2: Final regression

In [13]:
#First, let's model the amount donated from the existing data, but first we need to scale it and do some feature selection! 

In [14]:
#We can use a lot of the "splitting" work already done. 
#y is already created for the regression. 
#For X we just need to drop Target_B as it matches our target too closely. 

In [15]:
data_reg = data[data['TARGET_D']!=0]

In [16]:
y = data_reg['TARGET_D']
X = data_reg.drop(['TARGET_D'], axis = 1)

In [17]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# numerical_scaled = scaler.fit_transform(numericalX)

In [18]:
# pd.DataFrame(numerical_scaled).head()

In [19]:
# from sklearn.feature_selection import VarianceThreshold 
# var_threshold = 0.02
# sel = VarianceThreshold(threshold=(var_threshold))

# sel = sel.fit(numerical_scaled)
# temp = sel.transform(numerical_scaled)
# temp = pd.DataFrame(temp)
# print(numerical_scaled.shape)
# print(temp.shape)

In [20]:
# sel.variances_ > var_threshold
# sel.get_support()
# var_list = list(sel.get_support())
# list(zip(numerical.columns, var_list))
# [col[0] for col in zip(numerical.columns, var_list) if col[1] == False]

In [21]:
#Getting an output of the columns with low variance so I can chose to keep them or not. 
#Will keep IC1, AGE, MAXRAMNT,AVGGIFT,LASTGIFT,NGIFTALL as significant data. 